# Images
> A tutorial of fastpages for Jupyter notebooks.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/chart-preview.png

## Scaled Image

In [35]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
    private final String inDir = "ghtop_images/tmp/"; // location of images
    private final String outDir = "ghtop_images/tmp-out/";  // location of created files
    private String inFile;
    private String resizedFile;
    private String asciiFile;
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;

    // Constructor obtains attributes of picture
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.resizedFile = this.outDir + name + "." + ext;
        this.asciiFile = this.outDir + name + ".txt";
        this.setStats();
    }

    
    // An image contains metadata, namely size, width, and height
    public void setStats() {
        BufferedImage img;
        try {
            Path path = Paths.get(this.inFile);
            this.bytes = Files.size(path);
            img = ImageIO.read(new File(this.inFile));
            this.width = img.getWidth();
            this.height = img.getHeight();
        } catch (IOException e) {
        }
    }

    // Console print of data
    public void printStats(String msg) {
        System.out.println(msg + ": " + this.bytes + " " + this.width + "x" + this.height + "  " + this.inFile);
    }

    // Convert scaled image into buffered image
    public static BufferedImage convertToBufferedImage(Image img) {

        // Create a buffered image with transparency
        BufferedImage bi = new BufferedImage(
                img.getWidth(null), img.getHeight(null),
                BufferedImage.TYPE_INT_ARGB);

        // magic?
        Graphics2D graphics2D = bi.createGraphics();
        graphics2D.drawImage(img, 0, 0, null);
        graphics2D.dispose();

        return bi;
    }
    
    // Scale or reduce to "scale" percentage provided
    public void resize(int scale) {
        BufferedImage img = null;
        Image resizedImg = null;
        Image grayImg = null; 

        int width = (int) (this.width * (scale/40.0) + 0.5);
        int height = (int) (this.height * (scale/100.0) + 0.5);

        try {
            // read an image to BufferedImage for processing
            img = ImageIO.read(new File(this.inFile));  // set buffer of image data
            // create a new BufferedImage for drawing
            resizedImg = img.getScaledInstance(width, height, Image.SCALE_SMOOTH);
            grayImg = img.getScaledInstance(width, height, Image.SCALE_SMOOTH);
        } catch (IOException e) {
            return;
        }

        try {
            ImageIO.write(convertToBufferedImage(resizedImg), this.ext, new File(resizedFile));
        } catch (IOException e) {
            return;
        }
        
        this.inFile = this.resizedFile;  // use scaled file vs original file in Class
        this.setStats();
    }
    
    // convert every pixel to an ascii character (ratio does not seem correct)
    public void convertToAscii() {
        BufferedImage img = null;
        PrintWriter asciiPrt = null;
        FileWriter asciiWrt = null;

        try {
            File file = new File(this.asciiFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }

        try {
            asciiPrt = new PrintWriter(asciiWrt = new FileWriter(this.asciiFile, true));
        } catch (IOException e) {
            System.out.println("ASCII out file create error: " + e);
        }

        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }

        for (int i = 0; i < img.getHeight(); i++) {
            for (int j = 0; j < img.getWidth(); j++) {
                Color col = new Color(img.getRGB(j, i));
                double pixVal = (((col.getRed() * 0.30) + (col.getBlue() * 0.59) + (col.getGreen() * 0.11)));
                try {
                    asciiPrt.print(asciiChar(pixVal));
                    asciiPrt.flush();
                    asciiWrt.flush();
                } catch (Exception ex) {
                }
            } 
            try {
                asciiPrt.println("");
                asciiPrt.flush();
                asciiWrt.flush();
            } catch (Exception ex) {
            }
        }
    }

    // conversion table, there may be better out there ie https://www.billmongan.com/Ursinus-CS173-Fall2020/Labs/ASCIIArt
    public String asciiChar(double g) {
        String str = " ";
        if (g >= 240) {
            str = "";
        } else if (g >= 230) {
            str = "@";
        } else if (g >= 210) {
            str = "#";
        } else if (g >= 200) {
            str = "W";
        } else if (g >= 190) {
            str = "&";
        } else if (g >= 180) {
            str = "0";
        } else if (g >= 170) {
            str = "m";
        } else if (g >= 160) {
            str = "K";
        } else if (g >= 150) {
            str = "U";
        } else if (g >= 140) {
            str = "o";
        } else if (g >= 130) {
            str = "I";
        } else if (g >= 120) {
            str = "{";
        } else if (g >= 110) {
            str = "=";
        } else if (g >= 100) {
            str = ">";
        } else if (g >= 90) {
            str = "+";
        } else if (g >= 80) {
            str = "(";
        } else if (g >= 70) {
            str = "\\";
        } else if (g >= 60) {
            str = ";";
        } else if (g >= 50) {
            str = "~";
        } else if (g >= 40) {
            str = ":";
        } else if (g >= 20) {
            str = ".";
        } else {
            str = "`";
        }
        return str;
    }

    // tester/driver
    public static void main(String[] args) throws IOException {
        Pics monaLisa = new Pics("MonaLisa", "png");
        monaLisa.printStats("Original");
        monaLisa.resize(33);
        monaLisa.printStats("Scaled");
        monaLisa.convertToAscii();

        Pics pumpkin = new Pics("pumpkin", "png");
        pumpkin.printStats("Original");
        pumpkin.resize(33);
        pumpkin.printStats("Scaled");
        pumpkin.convertToAscii();
    }
}
Pics.main(null);

Original: 499298 389x413  ghtop_images/tmp/MonaLisa.png
Scaled: 139407 321x136  ghtop_images/tmp-out/MonaLisa.png
Original: 39392 302x265  ghtop_images/tmp/pumpkin.png
Scaled: 19531 249x87  ghtop_images/tmp-out/pumpkin.png


## Gray Scale

In [36]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
    private final String inDir = "ghtop_images/tmp/"; // location of images
    private final String outDir = "ghtop_images/tmp-out/";  // location of created files
    private String inFile;
    private String resizedFile;
    private String asciiFile;
    private String greyFile;
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;

    // Constructor obtains attributes of picture
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.resizedFile = this.outDir + name + "." + ext;
        this.asciiFile = this.outDir + name + ".txt";
        this.greyFile = this.outDir + name + ".png";
        this.setStats();
    }

    
    // An image contains metadata, namely size, width, and height
    public void setStats() {
        BufferedImage img;
        try {
            Path path = Paths.get(this.inFile);
            this.bytes = Files.size(path);
            img = ImageIO.read(new File(this.inFile));
            this.width = img.getWidth();
            this.height = img.getHeight();
        } catch (IOException e) {
        }
    }

    // Console print of data
    public void printStats(String msg) {
        System.out.println(msg + ": " + this.bytes + " " + this.width + "x" + this.height + "  " + this.inFile);
    }

    // Convert scaled image into buffered image
    public static BufferedImage convertToBufferedImage(Image img) {

        // Create a buffered image with transparency
        BufferedImage bi = new BufferedImage(
                img.getWidth(null), img.getHeight(null),
                BufferedImage.TYPE_INT_ARGB);

        // magic?
        Graphics2D graphics2D = bi.createGraphics();
        graphics2D.drawImage(img, 0, 0, null);
        graphics2D.dispose();

        return bi;
    }
    
    // Scale or reduce to "scale" percentage provided
    public void resize(int scale) {
        BufferedImage img = null;
        Image resizedImg = null;  

        int width = (int) (this.width * (scale/100.0) + 0.5);
        int height = (int) (this.height * (scale/100.0) + 0.5);

        try {
            img = ImageIO.read(new File(this.inFile));  // set buffer of image data
            // create a new BufferedImage for drawing
            resizedImg = img.getScaledInstance(width, height, Image.SCALE_SMOOTH);
        } catch (IOException e) {
            return;
        }

        try {
            ImageIO.write(convertToBufferedImage(resizedImg), this.ext, new File(resizedFile));
        } catch (IOException e) {
            return;
        }
        
        this.inFile = this.resizedFile;  
        this.setStats();
    }
    
    public void convertToGrey() {
        BufferedImage img = null;
        PrintWriter asciiPrt = null;
        FileWriter asciiWrt = null;

        try {
            File file = new File(this.asciiFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }
        try {
            asciiPrt = new PrintWriter(asciiWrt = new FileWriter(this.asciiFile, true));
        } catch (IOException e) {
            System.out.println("ASCII out file create error: " + e);
        }

        try {
            img = ImageIO.read(new File(this.inFile));
        } catch (IOException e) {
        }

        for (int i = 0; i < img.getHeight(); i++) {
            for (int j = 0; j < img.getWidth(); j++) {
                Color col = new Color(img.getRGB(j, i));
                // double pixVal = (((col.getRed() * 0.30) + (col.getBlue() * 0.59) + (col
                //         .getGreen() * 0.11)));
                int r = (int) (col.getRed() * 0.299);
                int g = (int) (col.getGreen() * 0.587);
                int b = (int) (col.getBlue() * 0.114);
                Color greyScale = new Color(
                            r + g + b,
                            r + g + b,
                            r + g + b);
                    img.setRGB(j, i, greyScale.getRGB());
            }
            try {
                asciiPrt.println("");
                asciiPrt.flush();
                asciiWrt.flush();
            } catch (Exception ex) {
            }
        }
        
        //write image
        try{
            File file = new File(this.greyFile);
            ImageIO.write(img, "png", file);
          }catch(IOException e){
            System.out.println(e);
          }
    }

    // conversion table, there may be better out there ie https://www.billmongan.com/Ursinus-CS173-Fall2020/Labs/ASCIIArt
    public String asciiChar(double g) {
        String str = " ";
        if (g >= 240) {
            str = " ";
        } else if (g >= 210) {
            str = ".";
        } else if (g >= 190) {
            str = "*";
        } else if (g >= 170) {
            str = "+";
        } else if (g >= 120) {
            str = "^";
        } else if (g >= 110) {
            str = "&";
        } else if (g >= 80) {
            str = "8";
        } else if (g >= 60) {
            str = "#";
        } else {
            str = "@";
        }
        return str;
    }

    // tester/driver
    public static void main(String[] args) throws IOException {
        Pics monaLisa = new Pics("MonaLisa", "png");
        // monaLisa.printStats("Original");
        // monaLisa.resize(33);
        // monaLisa.printStats("Scaled");
        monaLisa.convertToGrey();

        Pics pumpkin = new Pics("pumpkin", "png");
        // pumpkin.printStats("Original");
        // pumpkin.resize(33);
        // pumpkin.printStats("Scaled");
        pumpkin.convertToGrey();
    }
}
Pics.main(null);

In [41]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
     
    private final String inDir = "ghtop_images/tmp"; // location of images
    private final String outDir = "ghtop_images/tmp-out/";  // location of created files
    private String inFile;
    private String resizedFile;
    private String greenscaleFile; 
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;
    

    // Constructor obtains attributes of picture
     
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.greenscaleFile = this.outDir + name + "Green." + ext;
    }
   
 
    // convert every pixel to an ascii character (ratio does not seem correct)
    public void greenscale() {
        BufferedImage img; 
        
        try {
            File file = new File(this.greenscaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }

        
      

        try {
            img = ImageIO.read(new File(this.inFile));
            for (int i = 0; i < img.getHeight(); i++) {
                for (int j = 0; j < img.getWidth(); j++) {
                    Color col = new Color(img.getRGB(j, i));
                    
                    int green = (int)(col.getGreen());
                    Color rgb = new Color(0, green, 0);
       
                    img.setRGB(j, i, rgb.getRGB());
                 
                }
               
            }
             
            ImageIO.write(img, "png", new File(this.greenscaleFile));
        } catch (IOException e) {
        }

    }

  
    // tester/driver
    public static void main(String[] args) throws IOException {
        // Pics monaLisa = new Pics("MonaLisa", "png");
        Pics monaLisa = new Pics("MonaLisa", "png" );
        monaLisa.greenscale(); 

        
    }
}
Pics.main(null);

## Greenscale

In [43]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
     
    private final String inDir = "ghtop_images/tmp"; // location of images
    private final String outDir = "ghtop_images/tmp-out/";  // location of created files
    private String inFile;
    private String resizedFile;
    private String greenscaleFile; 
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;
    

    // Constructor obtains attributes of picture
     
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.greenscaleFile = this.outDir + name + "Green." + ext;
    }
   
 
    // convert every pixel to an ascii character (ratio does not seem correct)
    public void greenscale() {
        BufferedImage img; 
        
        try {
            File file = new File(this.greenscaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }

        
      

        try {
            img = ImageIO.read(new File(this.inFile));
            for (int i = 0; i < img.getHeight(); i++) {
                for (int j = 0; j < img.getWidth(); j++) {
                    Color col = new Color(img.getRGB(j, i));
                    
                    int green = (int)(col.getGreen());
                    Color rgb = new Color(0, green, 0);
       
                    img.setRGB(j, i, rgb.getRGB());
                 
                }
               
            }
             
            ImageIO.write(img, "png", new File(this.greenscaleFile));
        } catch (IOException e) {
        }

    }

  
    // tester/driver
    public static void main(String[] args) throws IOException {
        // Pics monaLisa = new Pics("MonaLisa", "png");
        Pics monaLisa = new Pics("MonaLisa1", "png" );
        monaLisa.greenscale(); 

        
    }
}
Pics.main(null);

## Red Scale

In [ ]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
     
    private final String inDir = "images/"; // location of images
    private final String outDir = "images/tmp/";  // location of created files
    private String inFile;
    private String resizedFile;
    private String redscaleFile; 
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;
    

    // Constructor obtains attributes of picture
     
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.redscaleFile = this.outDir + name + "Red." + ext;
    }
   
 
    // convert every pixel to an ascii character (ratio does not seem correct)
    public void redscale() {
        BufferedImage img; 
        
        try {
            File file = new File(this.redscaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }

        
      

        try {
            img = ImageIO.read(new File(this.inFile));
            for (int i = 0; i < img.getHeight(); i++) {
                for (int j = 0; j < img.getWidth(); j++) {
                    Color col = new Color(img.getRGB(j, i));
                    
                    int red = (int)(col.getRed());
                    Color rgb = new Color(red, 0, 0);
       
                    img.setRGB(j, i, rgb.getRGB());
                 
                }
               
            }
             
            ImageIO.write(img, "png", new File(this.redscaleFile));
        } catch (IOException e) {
        }

    }

  
    // tester/driver
    public static void main(String[] args) throws IOException {
        // Pics monaLisa = new Pics("MonaLisa", "png");
        Pics monaLisa = new Pics("MonaLisa", "png" );
        monaLisa.redscale(); 

        
    }
}
Pics.main(null);

## Blue Scale

In [ ]:
import java.awt.Color;
import java.awt.image.BufferedImage;
import java.awt.Image;
import java.awt.Graphics2D;

import java.io.File;
import java.io.FileWriter;
import java.io.IOException;
import java.io.PrintWriter;
import java.nio.file.Files;
import java.nio.file.Path;
import java.nio.file.Paths;

import javax.imageio.stream.ImageOutputStream;
import javax.imageio.stream.ImageInputStream;
import javax.imageio.metadata.IIOMetadata;
import javax.imageio.IIOImage;
import javax.imageio.ImageIO;
import javax.imageio.ImageWriteParam;
import javax.imageio.ImageWriter;
import javax.imageio.ImageReader;
import javax.imageio.ImageTypeSpecifier;

public class Pics {
     
    private final String inDir = "images/"; // location of images
    private final String outDir = "images/tmp/";  // location of created files
    private String inFile;
    private String resizedFile;
    private String bluescaleFile; 
    private String ext;   // extension of file
    private long bytes;
    private int width;
    private int height;
    

    // Constructor obtains attributes of picture
     
    public Pics(String name, String ext) {
        this.ext = ext;
        this.inFile = this.inDir + name + "." + ext;
        this.bluescaleFile = this.outDir + name + "Blue." + ext;
    }
   
 
    // convert every pixel to an ascii character (ratio does not seem correct)
    public void bluescale() {
        BufferedImage img; 
        
        try {
            File file = new File(this.bluescaleFile);
            Files.deleteIfExists(file.toPath()); 
        } catch (IOException e) {
            System.out.println("Delete File error: " + e);
        }

        
      

        try {
            img = ImageIO.read(new File(this.inFile));
            for (int i = 0; i < img.getHeight(); i++) {
                for (int j = 0; j < img.getWidth(); j++) {
                    Color col = new Color(img.getRGB(j, i));
                    
                    int blue = (int)(col.getBlue());
                    Color rgb = new Color(0, 0, blue);
       
                    img.setRGB(j, i, rgb.getRGB());
                 
                }
               
            }
             
            ImageIO.write(img, "png", new File(this.bluescaleFile));
        } catch (IOException e) {
        }

    }

  
    // tester/driver
    public static void main(String[] args) throws IOException {
        // Pics monaLisa = new Pics("MonaLisa", "png");
        Pics monaLisa = new Pics("MonaLisa", "png" );
        monaLisa.bluescale(); 

        
    }
}
Pics.main(null);

![](/home/vidhi/vscode/fastpages-CSA/_notebooks/ghtop_images/tmp-out/MonaLisaRed.png)

![](/home/vidhi/vscode/fastpages-CSA/_notebooks/ghtop_images/tmp-out/w6_img-monaLisa.jpg)

![](/home/vidhi/vscode/fastpages-CSA/_notebooks/ghtop_images/tmp-out/MonaLisaGreen.png)